In [1]:
import os
import PIL
import PIL.Image
import pathlib
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [2]:
# funkce zmensi obrazky na scale % sve puvodni velikosti
def downscale_pics(pics, scale=30):
    resized = []
    for img in pics:
         
        width = int(img.shape[1] * scale / 100)
        height = int(img.shape[0] * scale / 100)
        dim = (width, height)

        resized.append(cv2.resize(img, dim, interpolation = cv2.INTER_AREA))
    
    return resized


In [3]:
# nacte obrazky
def load_and_preprocess(path):
    
    # adresar s obrazy
    data_dir = pathlib.Path(path)

    # zobrazeni poctu obrazu v adresari
    image_count = len(list(data_dir.glob('*')))
    print(f"Celkem obrazů v adresáři {path}: {image_count}")


    paintings_path = list(data_dir.glob('*'))
    paintings_path = [str(path) for path in paintings_path]
    
    # ted pracuji pouze s sedymi obrazy
    paintings_gray = [cv2.imread(painting_path,  cv2.IMREAD_GRAYSCALE) for painting_path in paintings_path]
    paintings = paintings_gray
    
    # Zmenseni obrazku ma na mem datasetu velmi pozitivni vliv!
    paintings = downscale_pics(paintings, scale=10)
    
    return paintings


In [4]:
stolen_art_dir_path = "./mydata-stolen/"
paintings = load_and_preprocess(stolen_art_dir_path)

Celkem obrazů v adresáři ./mydata-stolen/: 9


# Data Augmentation

In [5]:
datagen = ImageDataGenerator()
gen = datagen.flow_from_directory(
    directory=stolen_art_dir_path,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)

Found 0 images belonging to 0 classes.


In [6]:
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

paintings = [np.array(p) for p in paintings]
paintings = [p.reshape(p.shape + (1,)) for p in paintings]
paintings = np.array(paintings)
print(paintings.shape)



painting0 = paintings[0]
painting0 = painting0.reshape((1,) + painting0.shape)
painting0 = painting0.reshape(painting0.shape + (1,))

datagen = ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


(9,)


In [7]:
#print(painting0.shape)
i = 0
for batch in datagen.flow(paintings, batch_size=1,
                          save_to_dir='preview', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

ValueError: setting an array element with a sequence.